In [1]:
import jax
import jax.numpy as jnp
from modulax.compound import Attention
from modulax.atom import Linear

In [2]:
num_heads = 5
d_model = 56
d_qk = 22
d_v = 26
b = 16
s = 24
x = jax.random.normal(jax.random.PRNGKey(0), (b, s, d_model))
attn = Attention(
    num_heads=num_heads,
    d_model=d_model,
    d_qk=d_qk,
    d_v=d_v,
)
p = attn.init_params(jax.random.PRNGKey(1))
o = attn.init_opt_state(jax.random.PRNGKey(2), p)
print(jax.tree.map(lambda x: x.shape, p))

((((110, 56), None), ((110, 56), None), ((130, 56), None)), (None, (None, (56, 130))))


In [4]:
y = attn(jax.random.PRNGKey(3), p, x)

In [5]:
def loss(p, x):
    y = attn(jax.random.PRNGKey(3), p, x)
    return jnp.mean(jnp.abs(y))

dp = jax.grad(loss)(p, x)
print(jax.tree.map(lambda x: x.shape, dp))

((((110, 56), None), ((110, 56), None), ((130, 56), None)), (None, (None, (56, 130))))


In [6]:
o, p_ = attn.apply_update(o, p, dp)
delta = jax.tree.map(lambda x, y: (x - y), p, p_)